### 1. Import libraries and dependencies

In [4]:
import copy
import itertools
import pickle
import time

import cv2 as cv
import mediapipe as mp
import numpy as np
import pandas as pd

### 2. Initialize constants and variables

In [5]:
# Initialize colors
black = (0, 0, 0)
grey_shade1 = (155, 168, 174)
grey_shade2 = (188, 202, 208)
grey_shade3 = (227, 232, 234)
white = (255, 255, 255)

# Initialize camera settings
webcam = 0
cap = cv.VideoCapture(webcam)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 960)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 540)

# Initialize misc.
init_prev_time = 0
escape_key = 27
press_key = 0xFF

# Initialize Mediapipe's hand model parameters
mp_hands = mp.solutions.hands
static_image_mode = False
max_num_hands = 1
min_detection_confidence = 0.8
min_tracking_confidence = 0.2
model_complexity = 0

hands = mp_hands.Hands(
    static_image_mode=static_image_mode,
    max_num_hands=max_num_hands,
    min_detection_confidence=min_detection_confidence,
    min_tracking_confidence=min_tracking_confidence,
    model_complexity=model_complexity
)

### 3. Initialize functions

##### Calculation functions

In [ ]:
def calc_bounding_box(image, landmarks):

    image_width, image_height = image.shape[1], image.shape[0]
    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv.boundingRect(landmark_array)

    return [x, y, x + w, y + h]

In [ ]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [ ]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

##### Cosmetic/Visual functions

In [ ]:
def draw_student_info(image):

    x_position = 323
    y_position = 470
    font_size = 0.3
    black_thickness = 2
    white_thickness = 1

    cv.putText(image, "* Achmad Mahathir P. (187006041) | Universitas Siliwangi 2022", (x_position, y_position),
               cv.FONT_HERSHEY_SIMPLEX, font_size,
               black, black_thickness,
               cv.LINE_AA)
    cv.putText(image, "* Achmad Mahathir P. (187006041) | Universitas Siliwangi 2022", (x_position, y_position),
               cv.FONT_HERSHEY_SIMPLEX, font_size,
               white, white_thickness,
               cv.LINE_AA)

    return image

In [ ]:
def draw_fps(image, fps):

    x_position = 10
    y_position = 30
    font_size = 0.73
    black_thickness = 4
    white_thickness = 2

    cv.putText(image, "FPS : " + str(int(fps)), (x_position, y_position), cv.FONT_HERSHEY_SIMPLEX, font_size, black,
               black_thickness,
               cv.LINE_AA)
    cv.putText(image, "FPS : " + str(int(fps)), (x_position, y_position), cv.FONT_HERSHEY_SIMPLEX, font_size, white,
               white_thickness,
               cv.LINE_AA)

    return image

In [ ]:
def draw_hand_detected(image):

    x_position = 10
    y_position = 60
    font_size = 0.73
    black_thickness = 4
    white_thickness = 2

    cv.putText(image, "Hand detected", (x_position, y_position), cv.FONT_HERSHEY_SIMPLEX, font_size,
               black, black_thickness, cv.LINE_AA)
    cv.putText(image, "Hand detected", (x_position, y_position), cv.FONT_HERSHEY_SIMPLEX, font_size,
               white, white_thickness, cv.LINE_AA)

    return image